In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets rouge-score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=37bf99d894d70a68eb0b4b5a2e91c6fb12aa8c1dcd0b2669a9295b4ed8742704
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")



In [ ]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub and get a sample
dataset = load_dataset('cnn_dailymail','3.0.0')
# sample = dataset['test'][randrange(len(dataset["test"]))]
test_set = dataset['test']
sample = dataset['test'].select(range(10))

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
test_set

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

## Different Approach (works)

In [ ]:
from re import VERBOSE

def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}

# -----------------------

# Use the map function to apply the generate_summary function to each sample in the dataset  <--- just for sample of 10
# summary_dataset = dataset.map(generate_summary, batched=True, batch_size=10)

# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = test_set.map(generate_summary, batched=True, batch_size=16)



# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test_set]

# -----------------------

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

KeyboardInterrupt: ignored

In [ ]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [ ]:
# Now compute ROUGE scores
scores = metric.compute(predictions=generated_summaries, references=reference_summaries)

# Output the scores
print(scores)

{'rouge1': AggregateScore(low=Score(precision=0.3952015987408577, recall=0.30457674466073287, fmeasure=0.3217059330219265), mid=Score(precision=0.39836998259304757, recall=0.3075511279235459, fmeasure=0.32418486483449), high=Score(precision=0.401647365643475, recall=0.3105221558141238, fmeasure=0.32673509240898124)), 'rouge2': AggregateScore(low=Score(precision=0.16431203696901348, recall=0.1297623537662084, fmeasure=0.13559621355686474), mid=Score(precision=0.16716023594188295, recall=0.1319598212032086, fmeasure=0.13781891266027968), high=Score(precision=0.16984752612072443, recall=0.13421255765543552, fmeasure=0.13998054748477645)), 'rougeL': AggregateScore(low=Score(precision=0.2916183788767058, recall=0.22220209217452125, fmeasure=0.23483961360150563), mid=Score(precision=0.2943795578192202, recall=0.2245617049910929, fmeasure=0.23683518849058072), high=Score(precision=0.2972551985925146, recall=0.22684147259007367, fmeasure=0.23890049207503417)), 'rougeLsum': AggregateScore(low=S

Results on full test set (11k)

'rouge1':

low=Score(precision=0.3952015987408577, recall=0.30457674466073287, fmeasure=0.3217059330219265),
mid=Score(precision=0.39836998259304757, recall=0.3075511279235459, fmeasure=0.32418486483449),
high=Score(precision=0.401647365643475, recall=0.3105221558141238, fmeasure=0.32673509240898124)

'rouge2':

low=Score(precision=0.16431203696901348, recall=0.1297623537662084, fmeasure=0.13559621355686474), mid=Score(precision=0.16716023594188295, recall=0.1319598212032086, fmeasure=0.13781891266027968),
high=Score(precision=0.16984752612072443, recall=0.13421255765543552, fmeasure=0.13998054748477645)

'rougeL':

low=Score(precision=0.2916183788767058, recall=0.22220209217452125, fmeasure=0.23483961360150563), mid=Score(precision=0.2943795578192202, recall=0.2245617049910929, fmeasure=0.23683518849058072),
high=Score(precision=0.2972551985925146, recall=0.22684147259007367, fmeasure=0.23890049207503417)),

'rougeLsum':

low=Score(precision=0.35266839743263423, recall=0.2696960142439835, fmeasure=0.2851648683201943), mid=Score(precision=0.35568242927726257, recall=0.2723172580752883, fmeasure=0.2876639379783771),
high=Score(precision=0.35872950016012195, recall=0.27488927672660946, fmeasure=0.2899468794168781))}


In [ ]:
#

On 1500 samples on Test Data

'rouge1':

AggregateScore(low=Score(precision=0.32094593602400817, recall=0.18791532107972814, fmeasure=0.21155197392385428), mid=Score(precision=0.3317754410226488, recall=0.19632160819085387, fmeasure=0.21890903222649116), high=Score(precision=0.3424986239070293, recall=0.204163233242157, fmeasure=0.22617384599344242)),

'rouge2':

AggregateScore(low=Score(precision=0.11009511175430851, recall=0.06748676934982654, fmeasure=0.07365364212458918), mid=Score(precision=0.11783089986535526, recall=0.07282269001813801, fmeasure=0.07896575144983646), high=Score(precision=0.1263589988607998, recall=0.07820229448814818, fmeasure=0.08420343243342436)),

'rougeL':

AggregateScore(low=Score(precision=0.2577356812709824, recall=0.14444508052024288, fmeasure=0.16402872549508932), mid=Score(precision=0.2674290262168536, recall=0.15095262770526263, fmeasure=0.1696379809596567), high=Score(precision=0.2775067665995306, recall=0.15766763741277406, fmeasure=0.17585746938590296)),

'rougeLsum':

AggregateScore(low=Score(precision=0.2898585336405672, recall=0.1668874997949044, fmeasure=0.18821504996956173), mid=Score(precision=0.3001239282203612, recall=0.17434532223661786, fmeasure=0.19513805251567945), high=Score(precision=0.31077342336012737, recall=0.18152374091505138, fmeasure=0.2015947669517604))}

On 500 samples from Test Data

'rouge-1': {'r': 0.12611584101750037, 'p': 0.27907001681550375, 'f': 0.15778487232777175},

'rouge-2': {'r': 0.03362445580264416, 'p': 0.07165524008702695, 'f': 0.040866743128156566},

'rouge-l': {'r': 0.11678755109220054, 'p': 0.2619427522230041, 'f': 0.14653107163491275}

On 100 samples from Test data

'rouge-1': {'r': 0.15206172281308006, 'p': 0.2835820020681796, 'f': 0.1825662688360879},

'rouge-2': {'r': 0.05303361151515525, 'p': 0.08499979667622784, 'f': 0.0605748768976535},

'rouge-l': {'r': 0.14112921218713304, 'p': 0.2643765077502626, 'f': 0.16917030020529353}

On 10 samples from Test Data

'rouge-1': {'r': 0.16800431065136948, 'p': 0.31554197994987476, 'f': 0.21156263367703274},

'rouge-2': {'r': 0.04879964750696458, 'p': 0.09698032961190856, 'f': 0.0625569178756907},

'rouge-l': {'r': 0.15942821354586062, 'p': 0.2927642021720969, 'f': 0.19934004231490648}

## BLEU score

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using Tesla T4


In [ ]:
model = model.to(device)

In [ ]:
def generate_summary(batch):
    # Ensure padding and truncation
    inputs = tokenizer(batch["article"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU

    # Generate summaries
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=120,
        do_sample=False,
        #top_p=0.9
    )
    outputs = outputs.cpu()  # Move outputs to CPU

    # Decode all generated summaries
    summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return {"summary": summaries}


In [ ]:
# Process the dataset with progress bar & increase batch_size for processing full test dataset
summary_dataset = test_set.map(generate_summary, batched=True, batch_size=16)



# Extract generated and reference summaries
generated_summaries = summary_dataset["summary"]
reference_summaries = [item["highlights"] for item in test_set]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
import sacrebleu


# Compute BLEU scores
bleu_score = sacrebleu.corpus_bleu(generated_summaries, [reference_summaries])
print(f'BLEU score: {bleu_score.score}')


BLEU score: 10.878674306175215


For 10 test cases: BLEU score: 3.745090969849481


For full test set: BLEU score: 10.878674306175215